# Imports

In [ ]:
import datetime
import itertools
import pickle
import sys

import xarray
import numpy
import matplotlib.pyplot as plt
from tensorflow import keras
import scipy.interpolate
import scipy.stats
import cmocean

In [ ]:
# IMD anomalies from 1901 to 2021, with climatology computed over 1950 to 2007
imd_regress_anomalies = xarray.open_dataarray('data/anomalies.nc')

In [ ]:
# ERA5 anomalies from 1950 to 2021, with climatology computed over 1993 to 2007
era5_verify_anomalies = xarray.open_dataarray('data/era5_anomalies.nc')
era5_clim = xarray.open_dataarray('data/era5_clim.nc')

In [ ]:
# SEAS5 anomalies from 1981 to 2021, with climatology computed over 1993 to 2007
seas5_anomalies = xarray.open_dataarray('data/seas5_anomalies.nc').isel(ensemble=range(25))

In [ ]:
seas5_clim = xarray.open_dataarray('data/seas5_clim.nc')

In [ ]:
mask = xarray.open_dataset('data/mask.h5')["mask"].values.astype(bool)
mask_lowres = xarray.open_dataset('data/mask_lowres.h5')["mask"].values.astype(bool)

In [ ]:
pcs = xarray.open_dataset('data/pcs.h5')["pcs"].values.T
pcs2 = pcs.reshape(107, 275, 2)

pcs_fcst = xarray.open_dataset('data/pcs_fcst.h5')["pcs"].values.T
pcs_fcst2 = pcs_fcst.reshape(9, 275, 2)

In [ ]:
nn_model_past = keras.models.load_model("data/nn_past")
nn_model_centered_era = keras.models.load_model("data/nn_centered_era")
nn_model_future_era = keras.models.load_model("data/nn_future_era")

In [ ]:
scaler_past = pickle.load(open('data/scaler_past', 'rb'))
scaler_centered_era = pickle.load(open('data/scaler_centered_era', 'rb'))
scaler_future_era = pickle.load(open('data/scaler_future_era', 'rb'))

# Compute MISO index

In [ ]:
miso_dat = era5_verify_anomalies.mean('lon').sel(lat=range(12, 32)).sel(time=(era5_verify_anomalies.time.dt.year <= 2008) & (era5_verify_anomalies.time.dt.year >= 1997) & (era5_verify_anomalies.time.dt.month >= 6) & (era5_verify_anomalies.time.dt.month <= 9)).values

In [ ]:
miso_dat2 = numpy.reshape(miso_dat, ((2008-1997+1), 122, 20))

In [ ]:
D = 20
n_regress_days = 15
n_days = 122
n_years = 2008 - 1997 + 1
n_samples = n_years*(n_days - n_regress_days)

x_new = numpy.zeros((n_samples, n_regress_days, D))

k = 0
for i in range(n_years):
    for j in range(n_regress_days, n_days):
        x_new[k, :, :] = miso_dat2[i, j-n_regress_days:j, :]
        k += 1

In [ ]:
x_new = numpy.reshape(x_new, (n_samples, -1))

In [ ]:
C = numpy.cov(x_new.T)

In [ ]:
u, s, v = numpy.linalg.svd(C)
eofs = v

# Definitions

In [ ]:
def running_mean(x, N):
    cumsum = numpy.cumsum(numpy.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
def forecast_pcs(initial_pcs, lead_time, k, year):
    pcs3 = pcs2[:year - year0, :, :]
    if lead_time == 0:
        return initial_pcs
    pc_dist = numpy.sqrt(numpy.sum((pcs3 - initial_pcs)**2, axis=2))
    idx2 = numpy.unravel_index(numpy.argsort(pc_dist.ravel()), pc_dist.shape)
    idx2 = (idx2[0][:k], idx2[1][:k])
    day_dists = (idx2[1][1:] - idx2[1][:-1])
    year_dists = (idx2[0][1:] - idx2[0][:-1])
    too_close = numpy.where((year_dists == 0) & (day_dists <= 10))[0] + 1
    too_close_mask = numpy.ones(idx2[1].shape, dtype=bool)
    too_close_mask[too_close] = False
    same_year = (idx2[1] + lead_time) < 275
    dist_mask = pc_dist[idx2[0], idx2[1]] <= pc_radius
    dists = pc_dist[idx2[0][same_year & too_close_mask & dist_mask], idx2[1][same_year & too_close_mask & dist_mask]]
    pcs_fcst = (pcs3[idx2[0][same_year & too_close_mask & dist_mask], idx2[1][same_year & too_close_mask & dist_mask] + lead_time, :]/dists.reshape(-1, 1)).sum(axis=0)/sum(1/dists.reshape(-1, 1))
    return pcs_fcst

In [ ]:
def bcorr(true, fcst):
    return sum(true[:, 0]*fcst[:, 0] + true[:, 1]*fcst[:, 1])/(numpy.sqrt(sum(true[:, 0]**2 + true[:, 1]**2))*numpy.sqrt(sum(fcst[:, 0]**2 + fcst[:, 1]**2)))

In [ ]:
def get_pcs_past(data):                                                              
    return nn_model_past(scaler_past.transform(data)).numpy()

def get_pcs_centered_era(data):                                                              
    return nn_model_centered_era(scaler_centered_era.transform(data)).numpy()

def get_pcs_future_era(data):                                                              
    return nn_model_future_era(scaler_future_era.transform(data)).numpy()

In [ ]:
def rmse(ensemble, obs):
    ens_mean = ensemble.mean(axis=0)
    return numpy.sqrt(numpy.nanmean((ens_mean - obs)**2))

def rmse_india(ensemble, obs):
    ens_mean = ensemble.mean(axis=0)
    return numpy.sqrt(numpy.nanmean((ens_mean[mask_lowres] - obs[mask_lowres])**2))

def corr(ensemble, obs):
    ens_mean = ensemble.mean(axis=0)
    return numpy.corrcoef(ens_mean.ravel(), obs.ravel())[0, 1]

In [ ]:
def run_enoc(year, month, lead_time, m_p, compute_mp=True, mp_metric=rmse):
    def opt(ordered_idx, test_ens, i):
        best_idx = ordered_idx[:i]
        return test_ens.sel(ensemble=best_idx)
    
    initial_date = datetime.datetime(year=year, month=month, day=day)
    
    if (m_p < n_ens) or compute_mp:
        initial_data = numpy.hstack([imd_regress_anomalies.sel(time=initial_date + datetime.timedelta(days=i)).values[mask] for i in range(-regress_days + 1, 1)])
        initial_pc = numpy.array(get_pcs_past(initial_data.reshape(1, -1)))

        date_days = (datetime.datetime(year=year, month=month, day=day) - datetime.datetime(year=year, month=3, day=1)).days

        pcs_fcst = forecast_pcs(initial_pc, lead_time, k=k, year=year)

        pcs_model = numpy.zeros((n_ens, n_pcs))

        for ens in range(n_ens):
            if lead_time <= half_regress_days:
                data_ens = [(seas5_anomalies.sel(initial_time=initial_date, ensemble=ens).isel(forecast_time=i-1).values).ravel() for i in range(lead_time, lead_time + regress_days)]
                pcs_model[ens, :] = numpy.array(get_pcs_future_era(numpy.hstack(data_ens).reshape(1, -1)))
            else:
                data_ens = [(seas5_anomalies.sel(initial_time=initial_date, ensemble=ens).isel(forecast_time=i-1).values).ravel() for i in range(lead_time - half_regress_days, lead_time + half_regress_days + 1)]
                pcs_model[ens, :] = numpy.array(get_pcs_centered_era(numpy.hstack(data_ens).reshape(1, -1)))

        if year >= 2008:
            true_pc = pcs_fcst2[year - 2008, date_days + lead_time, :]

        pc_dists = numpy.mean((pcs_model - pcs_fcst)**2, axis=1)

        if not any(numpy.isnan(pc_dists)):
            ordered_idx = numpy.argsort(pc_dists)
            best_idx = ordered_idx[:m_p]
        else:
            raise Exception
    else:
        best_idx = range(n_ens)

    enoc_ens = seas5_anomalies.sel(ensemble=best_idx, initial_time=initial_date).isel(forecast_time=range(lead_time - l_average_days - 1, lead_time + r_average_days)).mean("forecast_time")
    mean_ens = seas5_anomalies.sel(ensemble=range(n_ens), initial_time=initial_date).isel(forecast_time=range(lead_time - l_average_days - 1, lead_time + r_average_days)).mean("forecast_time")

    obs_state_era = numpy.mean([era5_verify_anomalies.sel(time=initial_date + datetime.timedelta(days=i)) for i in range(lead_time - l_average_days, lead_time + r_average_days + 1)], axis=0)
    obs_state_imd = numpy.mean([imd_regress_anomalies.sel(time=initial_date + datetime.timedelta(days=i)) for i in range(lead_time - l_average_days, lead_time + r_average_days + 1)], axis=0)
    
    if compute_mp:
        test_ens = seas5_anomalies.sel(initial_time=initial_date).isel(forecast_time=range(lead_time - l_average_days - 1, lead_time + r_average_days)).mean("forecast_time")

        opt_mp = [mp_metric(opt(ordered_idx, test_ens, i), obs_state_era) for i in range(1, n_ens + 1)]

    print("Year: ", year, "Month: ", month, "Lead time: ", lead_time)
    
    return {'ens_corr': enoc_ens, 'ens_uncorr': mean_ens, 'obs_state': obs_state_era, 'obs_state_imd': obs_state_imd, 'opt_mp': opt_mp if compute_mp else None,
            'pcs': (true_pc, pcs_fcst, pcs_model) if year >= 2008 else None}

In [ ]:
def run_pcs(year, month, day, lead_time):    
    initial_date = datetime.datetime(year=year, month=month, day=day)

    initial_data = numpy.hstack([imd_regress_anomalies.sel(time=initial_date + datetime.timedelta(days=i)).values[mask] for i in range(-regress_days + 1, 1)])
    initial_pc = numpy.array(get_pcs_past(initial_data.reshape(1, -1)))

    date_days = (datetime.datetime(year=year, month=month, day=day) - datetime.datetime(year=year, month=3, day=1)).days

    pcs_fcst = forecast_pcs(initial_pc, lead_time, k=k, year=year)

    if year >= 2008:
        true_pc = pcs_fcst2[year - 2008, date_days + lead_time, :]
    else:
        true_pc = pcs2[year - 1901, date_days + lead_time, :]

    #print("Year: ", year, "Month: ", month, "Day: ", day, "Lead time: ", lead_time)
    
    return (true_pc, pcs_fcst)

In [ ]:
import scipy.interpolate

highres_lons, highres_lats = numpy.meshgrid(numpy.arange(66.5, 100.25, 0.25),
                                            numpy.arange(6.5, 38.75, 0.25))

lowres_lons, lowres_lats = numpy.meshgrid(era5_verify_anomalies.lon, era5_verify_anomalies.lat)
lonnan, latnan = numpy.where(~mask)

def upscale(data):
    coords = numpy.vstack((highres_lons.ravel(), highres_lats.ravel())).T
    interp = scipy.interpolate.griddata(coords, data.ravel(), (lowres_lons, lowres_lats), method='nearest')

    return interp

def downscale(data):
    coords = numpy.vstack((lowres_lons.ravel(), lowres_lats.ravel())).T
    interp = scipy.interpolate.griddata(coords, data.ravel(), (highres_lons, highres_lats), method='cubic')
    interp[lonnan, latnan] = numpy.nan

    return interp

In [ ]:
# Constants
day = 1
regress_days = 29
half_regress_days = 14
regress_days_past = 45
n_ens = 25
n_pcs = 2
k = 50
pc_radius = 1.0
year0 = 1901

In [ ]:
# Load saved results
all_res_obs = pickle.load(open("all_res_obs_avg2", "rb"))
all_res_corr = pickle.load(open("all_res_corr_avg2", "rb"))
all_res_uncorr = pickle.load(open("all_res_uncorr_avg2", "rb"))
all_pcs = pickle.load(open("all_pcs", "rb"))

In [ ]:
def remappedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the median value of a colormap, and scale the
    remaining color range. Useful for data with a negative minimum and
    positive maximum where you want the middle of the colormap's dynamic
    range to be at zero.
    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower ofset). Should be between
          0.0 and 0.5; if your dataset mean is negative you should leave 
          this at 0.0, otherwise to (vmax-abs(vmin))/(2*vmax) 
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0; usually the
          optimal value is abs(vmin)/(vmax+abs(vmin)) 
      stop : Offset from highets point in the colormap's range.
          Defaults to 1.0 (no upper ofset). Should be between
          0.5 and 1.0; if your dataset mean is positive you should leave 
          this at 1.0, otherwise to (abs(vmin)-vmax)/(2*abs(vmin)) 
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.hstack([
        np.linspace(start, 0.5, 128, endpoint=False), 
        np.linspace(0.5, stop, 129)
    ])

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import matplotlib

vmin = 0.0
vmax = 0.25

np = numpy
cmap = matplotlib.colormaps['RdBu']
cmap = ListedColormap(cmap(np.linspace(0.2, 0.8, 256)))
cmap = remappedColorMap(cmap, start=(vmax-abs(vmin))/(2*vmax), midpoint=abs(vmin)/(vmax+abs(vmin)), stop=1.0, name='shrunk')
#levels = numpy.linspace(vmin, vmax, 17)

In [ ]:
import pandas
region_data = pandas.read_hdf('data/homo_regions.h5')

In [ ]:
regions = ['South_Peninsular', 'West_Central', 'Central_Northeast', 'Northwest', 'Northeast', 'Hilly_Regions']

# Misc. plots

In [ ]:
plt.plot(numpy.sqrt(pcs2.mean(axis=0)[:, 0]**2 + pcs2.mean(axis=0)[:, 1]**2))
plt.xticks([0, 31, 61, 92, 122, 153, 184, 214, 245],
           ['March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November'], rotation=30)
plt.xlabel('Day of year', size=14)
plt.ylabel('Average PC vector magnitude', size=14)
plt.savefig("pc_magnitude.pdf")

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

dat = region_data.copy()
dat[dat == 'South_Peninsular'] = 0
dat[dat == 'Hilly_Regions'] = 1
dat[dat == 'Central_Northeast'] = 2
dat[dat == 'West_Central'] = 3
dat[dat == 'Northeast'] = 4
dat[dat == 'Northwest'] = 5
m = numpy.zeros(lowres_lons.shape)*numpy.nan
m[mask_lowres] = dat
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')

plt.pcolormesh(lowres_lons, lowres_lats, m, cmap='Pastel2')
plt.text(numpy.mean(lowres_lons[mask_lowres][region_data == 'South_Peninsular'])-1,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'South_Peninsular'])-1, 'South\nPeninsular',
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
plt.text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Hilly_Regions'])-1.5,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Hilly_Regions']), 'Hilly Regions',
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
plt.text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Central_Northeast']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Central_Northeast']), 'Central\nNortheast',
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
plt.text(numpy.mean(lowres_lons[mask_lowres][region_data == 'West_Central']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'West_Central'])-1.5, 'West Central',
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
plt.text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Northeast']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Northeast']), 'Northeast',
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
plt.text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Northwest']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Northwest']-0.2), 'Northwest',
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
plt.savefig("india_regions_homo.pdf")

# EnOC

In [ ]:
m_p = 25

opt_mps = []

l_average_days = 7
r_average_days = 7

all_res_corr = {}
all_res_uncorr = {}
all_res_obs = {}
all_res_obs_imd = {}
all_res_obs_imd = {}
all_pcs = {}

for month in range(7, 10):
    all_res_corr[month] = {}
    all_res_uncorr[month] = {}
    all_res_obs[month] = {}
    all_res_obs_imd[month] = {}
    all_pcs[month] = {}

    for lead_time in range(0, 45):
        all_res_corr[month][lead_time] = []
        all_res_uncorr[month][lead_time] = []
        all_res_obs[month][lead_time] = []
        all_res_obs_imd[month][lead_time] = []
        all_pcs[month][lead_time] = []

        opt_mps = []

        for year in range(1993, 2008):
            res = run_enoc(year, month, lead_time, m_p, mp_metric=rmse)
            opt_mps.append(res['opt_mp'])

        m_p = numpy.argmin(numpy.mean(opt_mps, axis=0)) + 1

        for year in range(2008, 2017):
            res = run_enoc(year, month, lead_time, m_p, compute_mp=True, mp_metric=rmse)

            all_res_corr[month][lead_time].append(res['ens_corr'])
            all_res_uncorr[month][lead_time].append(res['ens_uncorr'])
            all_res_obs[month][lead_time].append(res['obs_state'])
            all_res_obs_imd[month][lead_time].append(res['obs_state_imd'])
            all_pcs[month][lead_time].append(res['pcs'])

In [ ]:
l_avg_days = 7
r_avg_days = 7

l_cont_days = 7
r_cont_days = 7

avg_corr = {}
avg_uncorr = {}
avg_obs = {}
avg_obs_imd = {}

for month in range(7, 10):
    avg_corr[month] = {}
    avg_uncorr[month] = {}
    avg_obs[month] = {}
    avg_obs_imd[month] = {}

    for lead_time in range(7, 38):
        avg_corr[month][lead_time] = []
        avg_uncorr[month][lead_time] = []
        avg_obs[month][lead_time] = []
        avg_obs_imd[month][lead_time] = []

        for year in range(9):
            corr_ens = list(set.intersection(*[set(all_res_corr[month][lead_time+i][year].ensemble.values) for i in range(-l_cont_days, r_cont_days+1)]))
            avg_corr[month][lead_time].append(numpy.mean([all_res_corr[month][lead_time+i][year].sel(ensemble=corr_ens) for i in range(-l_avg_days, r_avg_days+1)], axis=0))
            avg_uncorr[month][lead_time].append(numpy.mean([all_res_uncorr[month][lead_time+i][year] for i in range(-l_avg_days, r_avg_days+1)], axis=0))
            avg_obs[month][lead_time].append(numpy.mean([all_res_obs[month][lead_time+i][year] for i in range(-l_avg_days, r_avg_days+1)], axis=0))
            avg_obs_imd[month][lead_time].append(numpy.mean([upscale(all_res_obs_imd[month][lead_time+i][year]) for i in range(-l_avg_days, r_avg_days+1)], axis=0)[mask_lowres])

In [ ]:
all_miso_corr = {}
all_miso_uncorr = {}
all_miso_obs = {}

for month in range(7, 10):
    all_miso_corr[month] = {}
    all_miso_uncorr[month] = {}
    all_miso_obs[month] = {}

    for lead_time in range(1+15, 40):
        all_miso_corr[month][lead_time] = []
        all_miso_uncorr[month][lead_time] = []
        all_miso_obs[month][lead_time] = []
        
        for year_i, year in enumerate(range(2008, 2017)):
            corrs = numpy.hstack([all_res_corr[month][lead_i][year_i].mean('ensemble').mean('lon').sel(lat=range(12, 32)) for lead_i in range(lead_time-15, lead_time)])
            uncorrs = numpy.hstack([all_res_uncorr[month][lead_i][year_i].mean('ensemble').mean('lon').sel(lat=range(12, 32)) for lead_i in range(lead_time-15, lead_time)])
            obs = numpy.hstack([all_res_obs[month][lead_i][year_i].mean(axis=1)[6:26] for lead_i in range(lead_time-15, lead_time)])
            
            all_miso_corr[month][lead_time].append(corrs@eofs[:2, :].T)
            all_miso_uncorr[month][lead_time].append(uncorrs@eofs[:2, :].T)
            all_miso_obs[month][lead_time].append(obs@eofs[:2, :].T)

In [ ]:
pickle.dump(all_res_obs, open("data/all_res_obs", "wb"))
pickle.dump(all_res_corr, open("data/all_res_corr", "wb"))
pickle.dump(all_res_uncorr, open("data/all_res_uncorr", "wb"))
pickle.dump(all_pcs, open("data/all_pcs", "wb"))

# Analysis plots

In [ ]:
corr_sum = numpy.zeros(24)
uncorr_sum = numpy.zeros(24)

i = 0

for month in range(7, 9):
    corr_sum += 0.5*numpy.arctanh(numpy.array([bcorr(numpy.array(all_miso_obs[month][i]), numpy.array(all_miso_corr[month][i])) for i in range(22-6, 40)]))
    uncorr_sum += 0.5*numpy.arctanh(numpy.array([bcorr(numpy.array(all_miso_obs[month][i]), numpy.array(all_miso_uncorr[month][i])) for i in range(22-6, 40)]))
    
    i += 1

In [ ]:
plt.figure(figsize=(5, 3.5))

plt.plot(range(23-6, 41), numpy.tanh(uncorr_sum), label="Baseline")
plt.plot(range(23-6, 41), numpy.tanh(corr_sum), label="EnOC")

plt.xlabel('Lead time (days)', fontsize=14)
plt.ylabel('Bivariate correlation', fontsize=14)
plt.legend()
#plt.savefig("miso_skill_july_august2.pdf")

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(7, 2.5), sharex=True, sharey=False)
axs=axs.flatten()

corrs_sum = numpy.zeros(21)
uncorrs_sum = numpy.zeros(21)
clims_sum = numpy.zeros(21)

i = 0
for month in range(7, 10):
    corrs = [numpy.mean([rmse(avg_corr[month][lead_time][i], avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]
    uncorrs = [numpy.mean([rmse(avg_uncorr[month][lead_time][i], avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]
    clims = [numpy.mean([rmse(avg_uncorr[month][lead_time][i]*0, avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]

    corrs_sum += numpy.array(corrs)/3
    uncorrs_sum += numpy.array(uncorrs)/3
    clims_sum += numpy.array(clims)/3

axs[i].plot(range(10, 31), 1 - uncorrs_sum/clims_sum, label="Uncorrected")
axs[i].plot(range(10, 31), 1 - corrs_sum/clims_sum, label="Corrected")
axs[i].plot(range(10, 31), 1 - clims_sum/clims_sum, label="Climatology", c='black',
            linestyle='--')
print(numpy.mean(numpy.array(uncorrs)/numpy.array(corrs)))

axs[i].set_title('Monsoon region', fontsize=14)

i += 1

corrs_sum = numpy.zeros(21)
uncorrs_sum = numpy.zeros(21)
clims_sum = numpy.zeros(21)
for month in range(7, 10):
    corrs = [numpy.mean([rmse_india(avg_corr[month][lead_time][i], avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]
    uncorrs = [numpy.mean([rmse_india(avg_uncorr[month][lead_time][i], avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]
    clims = [numpy.mean([rmse_india(avg_uncorr[month][lead_time][i]*0, avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]

    corrs_sum += numpy.array(corrs)/3
    uncorrs_sum += numpy.array(uncorrs)/3
    clims_sum += numpy.array(clims)/3
    
axs[i].plot(range(10, 31), 1 - uncorrs_sum/clims_sum, label="Uncorrected")
axs[i].plot(range(10, 31), 1 - corrs_sum/clims_sum, label="Corrected")
axs[i].plot(range(10, 31), 1 - clims_sum/clims_sum, label="Climatology", c='black',
            linestyle='--')

axs[i].set_title('India', fontsize=14)

i += 1

axs[0].set_ylabel("RMSE skill score", fontsize=14)
axs[0].set_xlabel("Lead time (days)", fontsize=14)
axs[1].set_xlabel("Lead time (days)", fontsize=14)

axs[0].legend()
#plt.savefig('rmsess_averaged.pdf')

In [ ]:
corrs_sum = numpy.zeros(21)
uncorrs_sum = numpy.zeros(21)
clims_sum = numpy.zeros(21)

for month in range(7, 10):
    corrs = [numpy.mean([corr(avg_corr[month][lead_time][i], avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]
    uncorrs = [numpy.mean([corr(avg_uncorr[month][lead_time][i], avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]
    clims = [numpy.mean([corr(avg_uncorr[month][lead_time][i]*0, avg_obs[month][lead_time][i]) for i in range(9)]) for lead_time in range(9, 30)]

    corrs_sum += numpy.array(corrs)/3
    uncorrs_sum += numpy.array(uncorrs)/3
    clims_sum += numpy.array(clims)/3

plt.figure(figsize=(5, 3.5))
plt.plot(range(10, 31), uncorrs_sum, label="Uncorrected")
plt.plot(range(10, 31), corrs_sum, label="Corrected")
plt.xlabel("Lead time (days)", fontsize=14)
plt.ylabel("Anomaly correlation", fontsize=14)
plt.xticks(range(10, 32, 4))
plt.legend()
#plt.savefig("acorr2.pdf")

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(11, 5), sharex=True, sharey=False)
axs=axs.flatten()

i = 0
for month in range(7, 10):
    #plt.subplot(1, 3, i)
    tcorr_corr = [numpy.corrcoef([y.mean(axis=0).sum() for y in avg_corr[month][lead_time]], [y.sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]
    tcorr_uncorr = [numpy.corrcoef([y.mean(axis=0).sum() for y in avg_uncorr[month][lead_time]], [y.sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]
    axs[i].plot(range(10, 31), tcorr_uncorr)
    axs[i].plot(range(10, 31), tcorr_corr)
    #axs[i].set_ylim(-0.3, 0.9)
    if min(tcorr_uncorr) < 0:
        axs[i].axhline(0.0,color='black',ls='--')
    axs[i].set_title('Monsoon region, ' + ['July', 'August', 'September'][i], fontsize=14)
    print(numpy.mean(tcorr_uncorr[3:17]), numpy.mean(tcorr_corr[3:17]), numpy.mean(tcorr_corr[3:17])-numpy.mean(tcorr_uncorr[3:17]))
    i += 1

for month in range(7, 10):
    tcorr_corr = [numpy.corrcoef([y.mean(axis=0)[mask_lowres].sum() for y in avg_corr[month][lead_time]], [y[mask_lowres].sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]
    tcorr_uncorr = [numpy.corrcoef([y.mean(axis=0)[mask_lowres].sum() for y in avg_uncorr[month][lead_time]], [y[mask_lowres].sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]
    axs[i].plot(range(10, 31), tcorr_uncorr, label='Uncorrected')
    axs[i].plot(range(10, 31), tcorr_corr, label='EnOC')
    #axs[i].set_ylim(-0.3, 0.9)
    axs[i].set_xlabel("Lead time (days)", fontsize=14)
    if min(tcorr_uncorr) < 0:
        axs[i].axhline(0.0,color='black',ls='--')
    axs[i].set_title('India, ' + ['July', 'August', 'September'][i%3], fontsize=14)
    print(numpy.mean(tcorr_uncorr[3:17]), numpy.mean(tcorr_corr[3:17]), numpy.mean(tcorr_corr[3:17])-numpy.mean(tcorr_uncorr[3:17]))
    i += 1
    
axs[0].set_ylabel("Temporal correlation", fontsize=14)
axs[3].set_ylabel("Temporal correlation", fontsize=14)
axs[5].legend()

#plt.savefig("tcorr_panel5.pdf")

#fig.text(0.5, 0.04, 'common X', ha='center')
#fig.text(0.04, 0.5, 'common Y', va='center', rotation='vertical')

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(7, 2.5), sharex=True, sharey=False)
axs=axs.flatten()

corr_sum = numpy.zeros(30-9)
uncorr_sum = numpy.zeros(30-9)

i = 0
for month in range(7, 9):
    tcorr_corr = [numpy.corrcoef([y.mean(axis=0).sum() for y in avg_corr[month][lead_time]], [y.sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]
    tcorr_uncorr = [numpy.corrcoef([y.mean(axis=0).sum() for y in avg_uncorr[month][lead_time]], [y.sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]

    corr_sum += numpy.arctanh(numpy.array(tcorr_corr))/2
    uncorr_sum += numpy.arctanh(numpy.array(tcorr_uncorr))/2
axs[i].plot(range(10, 31), numpy.tanh(uncorr_sum), label='Uncorrected')
axs[i].plot(range(10, 31), numpy.tanh(corr_sum), label='EnOC')
#axs[i].set_ylim(0.52, 0.87)
axs[i].set_title('Monsoon region', fontsize=14)
axs[i].set_xlabel("Lead time (days)", fontsize=14)

#print(numpy.mean(tcorr_uncorr[3:17]), numpy.mean(tcorr_corr[3:17]), numpy.mean(tcorr_corr[3:17])-numpy.mean(tcorr_uncorr[3:17]))
i += 1

corr_sum = numpy.zeros(30-9)
uncorr_sum = numpy.zeros(30-9)

for month in range(7, 9):
    tcorr_corr = [numpy.corrcoef([y.mean(axis=0)[mask_lowres].sum() for y in avg_corr[month][lead_time]], [y[mask_lowres].sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]
    tcorr_uncorr = [numpy.corrcoef([y.mean(axis=0)[mask_lowres].sum() for y in avg_uncorr[month][lead_time]], [y[mask_lowres].sum() for y in avg_obs[month][lead_time]])[0, 1] for lead_time in range(9, 30)]

    corr_sum += numpy.arctanh(numpy.array(tcorr_corr))/2
    uncorr_sum += numpy.arctanh(numpy.array(tcorr_uncorr))/2
    
axs[i].plot(range(10, 31), numpy.tanh(uncorr_sum), label='Baseline')
axs[i].plot(range(10, 31), numpy.tanh(corr_sum), label='EnOC')
#axs[i].set_ylim(0.27, 0.92)
axs[i].set_xlabel("Lead time (days)", fontsize=14)
if min(tcorr_uncorr) < 0:
    axs[i].axhline(0.0,color='black',ls='--')
axs[i].set_title('India', fontsize=14)
#print(numpy.mean(tcorr_uncorr[3:17]), numpy.mean(tcorr_corr[3:17]), numpy.mean(tcorr_corr[3:17])-numpy.mean(tcorr_uncorr[3:17]))
i += 1
    
axs[0].set_ylabel("Temporal correlation", fontsize=14)
axs[1].legend()
#plt.savefig('tcorr_july_august.pdf')

In [ ]:
#IMD data

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(7, 2.5), sharex=True, sharey=False)
axs=axs.flatten()

corr_sum = numpy.zeros(30-9)
uncorr_sum = numpy.zeros(30-9)

i = 0
for month in range(7, 9):
    tcorr_corr = [numpy.corrcoef([y.mean(axis=0)[mask_lowres].sum() for y in avg_corr[month][lead_time]], [y.sum() for y in avg_obs_imd[month][lead_time]])[0, 1] for lead_time in range(9, 30)]
    tcorr_uncorr = [numpy.corrcoef([y.mean(axis=0)[mask_lowres].sum() for y in avg_uncorr[month][lead_time]], [y.sum() for y in avg_obs_imd[month][lead_time]])[0, 1] for lead_time in range(9, 30)]

    corr_sum += numpy.arctanh(numpy.array(tcorr_corr))/2
    uncorr_sum += numpy.arctanh(numpy.array(tcorr_uncorr))/2
axs[i].plot(range(10, 31), numpy.tanh(uncorr_sum), label='Uncorrected')
axs[i].plot(range(10, 31), numpy.tanh(corr_sum), label='EnOC')
#axs[i].set_ylim(0.52, 0.87)
axs[i].set_title('India (IMD observations)', fontsize=14)
axs[i].set_xlabel("Lead time (days)", fontsize=14)
axs[0].set_ylabel("Temporal correlation", fontsize=14)
#plt.savefig('tcorr_imd.pdf')

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

corrs_corr = dict((region, 0) for region in regions)
corrs_uncorr = dict((region, 0) for region in regions)
for month in range(7, 10):
    #plt.subplot(3, 2, i)

    for region in regions:
        ensembles_corr = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_corr[month][lead_time]] for lead_time in range(13, 20)]),
                                   axis=0)
        ensembles_uncorr = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_uncorr[month][lead_time]] for lead_time in range(13, 20)]),
                                   axis=0)
        observations = numpy.sum(numpy.array([[y[mask_lowres][region_data == region].sum() for y in avg_obs[month][lead_time]] for lead_time in range(13, 20)]),
                                 axis=0)
        corrs_corr[region] += numpy.arctanh(numpy.corrcoef(ensembles_corr, observations)[0, 1])/3
        corrs_uncorr[region] += numpy.arctanh(numpy.corrcoef(ensembles_uncorr, observations)[0, 1])/3

fig, axs = plt.subplots(nrows=1, ncols=2,
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(11, 8.5))
axs=axs.T.flatten()

import cmocean

dat = region_data.copy()
for region in regions:
    dat[dat == region] = numpy.tanh(corrs_corr[region]) - numpy.tanh(corrs_uncorr[region])
    print(region, numpy.tanh(corrs_corr[region]) - numpy.tanh(corrs_uncorr[region]))

m = numpy.zeros(lowres_lons.shape)*numpy.nan
m[mask_lowres] = dat
#m[~mask_lowres] = corrs_corr['null'] - corrs_uncorr['null']
#ax = plt.axes(projection=ccrs.PlateCarree())

#z = m[:-1, :-1]
#levels = MaxNLocator(nbins=50).tick_values(-numpy.nanmax(z) - 0.2, numpy.nanmax(z) + 0.2)


# pick the desired colormap, sensible levels, and define a normalization
# instance which takes data values and translates those into levels.
#cmap = cmocean.cm.balance_r
#norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

#ax = plt.axes(projection=ccrs.LambertCylindrical())
cs = axs[0].pcolormesh(lowres_lons, lowres_lats, m, cmap=cmap, vmin=vmin, vmax=vmax)#, norm=divnorm)
axs[0].coastlines()
axs[0].add_feature(cfeature.BORDERS, linestyle=':')

axs[0].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'South_Peninsular'])-1,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'South_Peninsular'])-1.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['South_Peninsular']) - numpy.tanh(corrs_uncorr['South_Peninsular'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[0].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Hilly_Regions'])-3.5,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Hilly_Regions'])+0.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['Hilly_Regions']) - numpy.tanh(corrs_uncorr['Hilly_Regions'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[0].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Central_Northeast'])+1.5,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Central_Northeast']), '+{:.2f}'.format(numpy.tanh(corrs_corr['Central_Northeast']) - numpy.tanh(corrs_uncorr['Central_Northeast'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[0].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'West_Central']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'West_Central'])-0.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['West_Central']) - numpy.tanh(corrs_uncorr['West_Central'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[0].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Northeast'])+1.5,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Northeast'])+0.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['Northeast']) - numpy.tanh(corrs_uncorr['Northeast'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[0].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Northwest']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Northwest']-0.2), '+{:.2f}'.format(numpy.tanh(corrs_corr['Northwest']) - numpy.tanh(corrs_uncorr['Northwest'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())

axs[0].set_title('Days 14–20', fontsize=20)

corrs_corr = dict((region, 0) for region in regions)
corrs_uncorr = dict((region, 0) for region in regions)
for month in range(7, 10):
    #plt.subplot(3, 2, i)

    for region in regions:
        ensembles_corr = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_corr[month][lead_time]] for lead_time in range(20, 27)]),
                                   axis=0)
        ensembles_uncorr = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_uncorr[month][lead_time]] for lead_time in range(20, 27)]),
                                   axis=0)
        observations = numpy.sum(numpy.array([[y[mask_lowres][region_data == region].sum() for y in avg_obs[month][lead_time]] for lead_time in range(20, 27)]),
                                 axis=0)
        corrs_corr[region] += numpy.arctanh(numpy.corrcoef(ensembles_corr, observations)[0, 1])/3
        corrs_uncorr[region] += numpy.arctanh(numpy.corrcoef(ensembles_uncorr, observations)[0, 1])/3

dat = region_data.copy()
for region in regions:
    dat[dat == region] = numpy.tanh(corrs_corr[region]) - numpy.tanh(corrs_uncorr[region])
    print(region, numpy.tanh(corrs_corr[region]) - numpy.tanh(corrs_uncorr[region]))

m = numpy.zeros(lowres_lons.shape)*numpy.nan
m[mask_lowres] = dat
#m[~mask_lowres] = corrs_corr['null'] - corrs_uncorr['null']
#ax = plt.axes(projection=ccrs.PlateCarree())

#ax = plt.axes(projection=ccrs.LambertCylindrical())
cs = axs[1].pcolormesh(lowres_lons, lowres_lats, m, cmap=cmap, vmin=vmin, vmax=vmax)#, norm=divnorm)
axs[1].coastlines()
axs[1].add_feature(cfeature.BORDERS, linestyle=':')

axs[1].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'South_Peninsular'])-1,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'South_Peninsular'])-1.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['South_Peninsular']) - numpy.tanh(corrs_uncorr['South_Peninsular'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[1].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Hilly_Regions'])-3.5,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Hilly_Regions'])+0.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['Hilly_Regions']) - numpy.tanh(corrs_uncorr['Hilly_Regions'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[1].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Central_Northeast'])+1.5,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Central_Northeast']), '+{:.2f}'.format(numpy.tanh(corrs_corr['Central_Northeast']) - numpy.tanh(corrs_uncorr['Central_Northeast'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[1].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'West_Central']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'West_Central'])-0.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['West_Central']) - numpy.tanh(corrs_uncorr['West_Central'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[1].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Northeast'])+1.5,
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Northeast'])+0.5, '+{:.2f}'.format(numpy.tanh(corrs_corr['Northeast']) - numpy.tanh(corrs_uncorr['Northeast'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())
axs[1].text(numpy.mean(lowres_lons[mask_lowres][region_data == 'Northwest']),
         numpy.mean(lowres_lats[mask_lowres][region_data == 'Northwest']-0.2), '+{:.2f}'.format(numpy.tanh(corrs_corr['Northwest']) - numpy.tanh(corrs_uncorr['Northwest'])),
         horizontalalignment='center',
         transform=ccrs.PlateCarree())

axs[1].set_title('Days 21–27', fontsize=20)

# m2 = numpy.zeros(lowres_lons.shape)
# dat = state_data["st_nm"].copy()
# dat[dat != 'South'] = 0
# dat[dat == 'South'] = 1
# m2[mask_lowres] = dat.values
# plt.pcolor(lowres_lons, lowres_lats, ma.masked_array(m2, mask=m2), facecolor='red')

#fig.colorbar(boundaries=numpy.linspace(-0.015, 0.085, 21))
import matplotlib as mpl
#norm = mpl.colors.BoundaryNorm(boundaries=[-0.015, 0.085], ncolors=50)
#mpl.colors.Normalize(vmin=vmin, vmax=vmax)

cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])
#cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap,
#                                norm=norm,
#                                orientation='horizontal')
cbar=fig.colorbar(cs, cax=cbar_ax, orientation='horizontal',
                  ticks=sorted(numpy.concatenate([numpy.arange(vmin, vmax + 0.01, 0.05), [0]])))
#cbar.ax.set_xticklabels(['-0.015', '-0.005', '', '0.005', '0.015', '0.025', '0.035', '0.045', '0.055', '0.065', '0.075', '0.085'])
cbar.set_label("$r_\\mathrm{corr}$ - $r_\\mathrm{uncorr}$", size=20)
#cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.tick_params(labelsize=12)
plt.savefig("tcorr_spatial_homo.pdf")

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import matplotlib

vmin = 0.0
vmax = 0.15

np = numpy
cmap = matplotlib.colormaps['RdBu']
cmap = ListedColormap(cmap(np.linspace(0.2, 0.8, 256)))
cmap = remappedColorMap(cmap, start=(vmax-abs(vmin))/(2*vmax), midpoint=abs(vmin)/(vmax+abs(vmin)), stop=1.0, name='shrunk2')
#levels = numpy.linspace(vmin, vmax, 17)

In [ ]:
for i, month in enumerate(range(7, 10)):
    bcorrs = [bcorr(numpy.array([all_pcs[month][l][y][0] for y in range(9)]), numpy.array([all_pcs[month][l][y][1] for y in range(9)]).reshape(9, 2)) for l in range(45)]
    plt.plot(range(1, 46), bcorrs, label=["July", "August", "September"][i%3])
    print(bcorrs[-1])

plt.title("Data-driven MISO forecast skill", fontsize=14)
plt.xlabel("Lead time (days)", fontsize=14)
plt.ylabel("Bivariate correlation", fontsize=14)
plt.legend()
#plt.savefig("miso_dd_skill.pdf")

In [ ]:
rmses = [rmse(avg_uncorr[m][l][y], avg_obs[m][l][y]) - rmse(avg_corr[m][l][y], avg_obs[m][l][y]) for m in range(9, 10) for l in range(7, 38) for y in range(9)]

In [ ]:
pc_errs = [numpy.sqrt(numpy.mean((all_pcs[m][l][y][0] - numpy.mean(all_pcs[m][l][y][2], axis=0))**2)) - numpy.sqrt(numpy.mean((all_pcs[m][l][y][0] - all_pcs[m][l][y][1])**2)) for m in range(9, 10) for l in range(7, 38) for y in range(9)]

In [ ]:
scipy.stats.pearsonr(pc_errs, rmses)

# Significance testing

In [ ]:
from random import choices

In [ ]:
# Bootstrap significance
corr_am = {}
uncorr_am = {}
obs_am = {}

for region in regions:
    corr_am[region] = {}
    uncorr_am[region] = {}
    obs_am[region] = {}
    for month in range(7, 10):
            corr_am[region][month] = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_corr[month][lead_time]] for lead_time in range(13, 20)]),
                                       axis=0)
            uncorr_am[region][month] = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_uncorr[month][lead_time]] for lead_time in range(13, 20)]),
                                       axis=0)
            obs_am[region][month] = numpy.sum(numpy.array([[y[mask_lowres][region_data == region].sum() for y in avg_obs[month][lead_time]] for lead_time in range(13, 20)]),
                                     axis=0)

for region in regions:
    print(region)
    samples_corr = []
    samples_uncorr = []
    for sample in range(10000):
        corr_corr = 0
        corr_uncorr = 0
        for month in range(7, 10):
            ensembles_corr = corr_am[region][month]
            ensembles_uncorr = uncorr_am[region][month]
            observations = obs_am[region][month]
            
            bootstrap_idx = choices(range(9), k=9)
            corr_corr += numpy.corrcoef(ensembles_corr[bootstrap_idx], observations[bootstrap_idx])[0, 1]/3
            corr_uncorr += numpy.corrcoef(ensembles_uncorr[bootstrap_idx], observations[bootstrap_idx])[0, 1]/3
        samples_corr.append(corr_corr)
        samples_uncorr.append(corr_uncorr)
    print(numpy.percentile(numpy.array(samples_corr) - numpy.array(samples_uncorr), (10, 15)))

for region in regions:
    corr_am[region] = {}
    uncorr_am[region] = {}
    obs_am[region] = {}
    for month in range(7, 10):
            corr_am[region][month] = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_corr[month][lead_time]] for lead_time in range(20, 27)]),
                                       axis=0)
            uncorr_am[region][month] = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres][region_data == region].sum() for y in avg_uncorr[month][lead_time]] for lead_time in range(20, 27)]),
                                       axis=0)
            obs_am[region][month] = numpy.sum(numpy.array([[y[mask_lowres][region_data == region].sum() for y in avg_obs[month][lead_time]] for lead_time in range(20, 27)]),
                                     axis=0)

for region in regions:
    print(region)
    samples_corr = []
    samples_uncorr = []
    for sample in range(10000):
        corr_corr = 0
        corr_uncorr = 0
        for month in range(7, 10):
            ensembles_corr = corr_am[region][month]
            ensembles_uncorr = uncorr_am[region][month]
            observations = obs_am[region][month]
            
            bootstrap_idx = choices(range(9), k=9)
            corr_corr += numpy.corrcoef(ensembles_corr[bootstrap_idx], observations[bootstrap_idx])[0, 1]/3
            corr_uncorr += numpy.corrcoef(ensembles_uncorr[bootstrap_idx], observations[bootstrap_idx])[0, 1]/3
        samples_corr.append(corr_corr)
        samples_uncorr.append(corr_uncorr)
    print(numpy.percentile(numpy.array(samples_corr) - numpy.array(samples_uncorr), (10, 15)))

In [ ]:
corr_am = {}
uncorr_am = {}
obs_am = {}

for lead_time in range(9, 30):
    corr_am[lead_time] = {}
    uncorr_am[lead_time] = {}
    obs_am[lead_time] = {}

    for month in range(7, 9):
        corr_am[lead_time][month] = numpy.array([y.mean(axis=0).sum() for y in avg_corr[month][lead_time]])
        uncorr_am[lead_time][month] = numpy.array([y.mean(axis=0).sum() for y in avg_uncorr[month][lead_time]])
        obs_am[lead_time][month] = numpy.array([y.sum() for y in avg_obs[month][lead_time]])

samples_corr = []
samples_uncorr = []
for lead_time in range(9, 30):
    samples_corr_lead = []
    samples_uncorr_lead = []

    for sample in range(10000):
        bootstrap_idx = choices(range(9), k=9)

        corr_sum = 0
        uncorr_sum = 0
        for month in range(7, 9):
            tcorr_corr = numpy.corrcoef(corr_am[lead_time][month][bootstrap_idx],
                                        obs_am[lead_time][month][bootstrap_idx])[0, 1]
            tcorr_uncorr = numpy.corrcoef(uncorr_am[lead_time][month][bootstrap_idx],
                                          obs_am[lead_time][month][bootstrap_idx])[0, 1]

            corr_sum += tcorr_corr/2
            uncorr_sum += tcorr_uncorr/2
        samples_corr_lead.append(corr_sum)
        samples_uncorr_lead.append(uncorr_sum)
    samples_corr.append(samples_corr_lead)
    samples_uncorr.append(samples_uncorr_lead)

In [ ]:
corr_am = {}
uncorr_am = {}
obs_am = {}

corr_am = {}
uncorr_am = {}
obs_am = {}
for month in range(7, 10):
        corr_am[month] = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres].sum() for y in avg_corr[month][lead_time]] for lead_time in range(9, 30)]),
                                   axis=0)
        uncorr_am[month] = numpy.sum(numpy.array([[y.mean(axis=0)[mask_lowres].sum() for y in avg_uncorr[month][lead_time]] for lead_time in range(9, 30)]),
                                   axis=0)
        obs_am[month] = numpy.sum(numpy.array([[y[mask_lowres].sum() for y in avg_obs[month][lead_time]] for lead_time in range(9, 30)]),
                                 axis=0)

samples_corr = []
samples_uncorr = []
for sample in range(10000):
    corr_corr = 0
    corr_uncorr = 0
    for month in range(7, 10):
        ensembles_corr = corr_am[month]
        ensembles_uncorr = uncorr_am[month]
        observations = obs_am[month]

        bootstrap_idx = choices(range(9), k=9)
        corr_corr += numpy.corrcoef(ensembles_corr[bootstrap_idx], observations[bootstrap_idx])[0, 1]/3
        corr_uncorr += numpy.corrcoef(ensembles_uncorr[bootstrap_idx], observations[bootstrap_idx])[0, 1]/3
    samples_corr.append(corr_corr)
    samples_uncorr.append(corr_uncorr)
print(numpy.percentile(numpy.array(samples_corr) - numpy.array(samples_uncorr), (5, 15)))